# wheelchair Isochrone

In [1]:
import json
import time
import os
import geopandas as gpd
from openrouteservice import Client
from openrouteservice.exceptions import ApiError

def calculate_isochrones_wheelchair(input_file, output_path, output_filename, print_statement):
    # Load poi data with GeoPandas
    poi = gpd.read_file(input_file)

    # Calculate centroids for each polygon
    poi['centroid'] = poi['geometry'].centroid

    # Change CRS to EPSG:4326
    poi = poi.to_crs("EPSG:4326")

    # Get the name of the ID column
    poi_id_column = 'id'  # Assuming the column name is 'id'

    # Initialize OpenRouteService client
    api_key = "5b3ce3597851110001cf6248ec8fc0286b4a4fd1a90fc8108327d823"  # Replace with your actual API key
    client = Client(key=api_key)

    # Define parameters for isochrone calculation
    profile = 'wheelchair'
    range_type = 'time'
    range_seconds = [900, 1800]  # 15 and 30 Minutes

    # Create the directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)

    # Function to calculate isochrones with retry mechanism
    def calculate_isochrones_with_retry(coordinate):
        retries = 3  # Number of retries
        for attempt in range(retries):
            try:
                isochrones = client.isochrones(locations=[coordinate], profile=profile, range_type=range_type, range=range_seconds, attributes=['total_pop'])
                return isochrones
            except ApiError as e:
                if "rate limit exceeded" in str(e):
                    print(f"Rate limit exceeded. Retrying in 10 seconds...")
                    time.sleep(10)  # Retry after 10 seconds for rate limit errors
                elif "Unable to build an isochrone map" in str(e):
                    print(f"Unable to build an isochrone map for POI. Skipping...")
                    return None
                else:
                    raise e  # Re-raise the exception if it's not related to rate limit exceeded
        return None

    # Iterate over poi and calculate isochrones
    for idx, row in poi.iterrows():
        # Get centroid coordinates of the poi
        lon, lat = row['centroid'].x, row['centroid'].y
        coordinate = (lon, lat)

        # Calculate isochrones with retry mechanism
        isochrones = calculate_isochrones_with_retry(coordinate)
        if isochrones is None:
            continue  # Skip the poi if isochrones cannot be calculated
            
        # Get the poi ID from the column
        poi_id = row[poi_id_column]

        # Add poi ID to each isochrone feature
        for feature in isochrones['features']:
            feature['properties']['poi_id'] = poi_id

        # Save isochrones data to file
        output_file_path = os.path.join(output_path, f'{output_filename}_{idx + 1}.geojson')
        
        with open(output_file_path, 'w') as output_file:
            json.dump(isochrones, output_file)

        print(print_statement.format(idx))

        # Rate limit handling
        time.sleep(3)  # Add a small delay to avoid rate limits and respect the rate limit

    print("Isochrone calculation completed.")


## Green spaces
### Barcelona

definitive wheelchair access

In [4]:
input_file = '../data/output/barcelona/access/wheelchair_greenspace.geojson'
output_path = '../data/output/barcelona/isochrones_wheelchair_greenspaces/definitive/'
output_filename = 'b_iso_wheelchair_greenspaces_d'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_wheelchair(input_file, output_path, output_filename, print_statement)

/var/folders/_n/_djf18r57q3482fkc8bbytk80000gn/T/ipykernel_52315/1740222986.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  poi['centroid'] = poi['geometry'].centroid


Isochrones calculated for POI 0
Isochrones calculated for POI 1
Isochrones calculated for POI 2
Isochrones calculated for POI 3
Isochrones calculated for POI 4
Isochrones calculated for POI 5
Isochrones calculated for POI 6
Isochrones calculated for POI 7
Isochrones calculated for POI 8
Isochrones calculated for POI 9
Isochrones calculated for POI 10
Isochrones calculated for POI 11
Isochrones calculated for POI 12
Isochrones calculated for POI 13
Isochrones calculated for POI 14
Isochrones calculated for POI 15
Isochrones calculated for POI 16
Isochrones calculated for POI 17
Isochrones calculated for POI 18
Isochrones calculated for POI 19
Isochrones calculated for POI 20
Isochrones calculated for POI 21
Isochrones calculated for POI 22
Isochrones calculated for POI 23
Isochrones calculated for POI 24
Isochrones calculated for POI 25
Isochrones calculated for POI 26
Isochrone calculation completed.


with ramps

In [5]:
input_file = '../data/output/barcelona/access/combined_greenspaces_wheelchair.geojson'
output_path = '../data/output/barcelona/isochrones_wheelchair_greenspaces/might/'
output_filename = 'b_iso_wheelchair_greenspaces_m'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_wheelchair(input_file, output_path, output_filename, print_statement)

/var/folders/_n/_djf18r57q3482fkc8bbytk80000gn/T/ipykernel_52315/1740222986.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  poi['centroid'] = poi['geometry'].centroid


Isochrones calculated for POI 0
Isochrones calculated for POI 1
Isochrones calculated for POI 2
Isochrones calculated for POI 3
Isochrones calculated for POI 4
Isochrones calculated for POI 5
Isochrones calculated for POI 6
Isochrones calculated for POI 7
Isochrones calculated for POI 8
Isochrones calculated for POI 9
Isochrones calculated for POI 10
Isochrones calculated for POI 11
Isochrones calculated for POI 12
Isochrones calculated for POI 13
Isochrones calculated for POI 14
Isochrones calculated for POI 15
Isochrones calculated for POI 16
Isochrones calculated for POI 17
Isochrones calculated for POI 18
Isochrones calculated for POI 19
Isochrones calculated for POI 20
Isochrones calculated for POI 21
Isochrones calculated for POI 22
Isochrones calculated for POI 23
Isochrones calculated for POI 24
Isochrones calculated for POI 25
Isochrones calculated for POI 26
Isochrones calculated for POI 27
Isochrones calculated for POI 28
Isochrones calculated for POI 29
Isochrones calculate

/Users/celinathome/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/celinathome/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/celinathome/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/celinathome/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/celinatho

Isochrones calculated for POI 61
Isochrones calculated for POI 62
Isochrones calculated for POI 63
Isochrones calculated for POI 64
Isochrones calculated for POI 65
Isochrones calculated for POI 66
Isochrones calculated for POI 67
Isochrones calculated for POI 68
Isochrones calculated for POI 69
Isochrones calculated for POI 70
Unable to build an isochrone map for POI. Skipping...
Isochrones calculated for POI 72
Isochrones calculated for POI 73
Isochrones calculated for POI 74
Isochrones calculated for POI 75
Isochrones calculated for POI 76
Isochrones calculated for POI 77
Isochrones calculated for POI 78
Isochrones calculated for POI 79
Isochrones calculated for POI 80
Isochrones calculated for POI 81
Isochrones calculated for POI 82
Isochrones calculated for POI 83
Isochrones calculated for POI 84
Isochrones calculated for POI 85
Isochrones calculated for POI 86
Isochrones calculated for POI 87
Isochrones calculated for POI 88
Isochrones calculated for POI 89
Isochrones calculated 

### Munich

In [7]:
input_file = '../data/output/munich/access/wheelchair_greenspace.geojson'
output_path = '../data/output/munich/isochrones_wheelchair_greenspaces/definitive/'
output_filename = 'm_iso_wheelchair_greenspaces_d'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_wheelchair(input_file, output_path, output_filename, print_statement)

input_file = '../data/output/munich/access/combined_greenspaces_wheelchair.geojson'
output_path = '../data/output/munich/isochrones_wheelchair_greenspaces/might/'
output_filename = 'm_iso_wheelchair_greenspaces_m'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_wheelchair(input_file, output_path, output_filename, print_statement)

/var/folders/_n/_djf18r57q3482fkc8bbytk80000gn/T/ipykernel_52315/1740222986.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  poi['centroid'] = poi['geometry'].centroid


Isochrones calculated for POI 0
Isochrones calculated for POI 1
Isochrones calculated for POI 2
Isochrones calculated for POI 3
Isochrones calculated for POI 4
Isochrones calculated for POI 5
Isochrones calculated for POI 6
Isochrones calculated for POI 7
Isochrones calculated for POI 8
Isochrones calculated for POI 9
Isochrones calculated for POI 10
Isochrones calculated for POI 11
Isochrones calculated for POI 12
Isochrones calculated for POI 13
Isochrones calculated for POI 14
Isochrones calculated for POI 15
Isochrones calculated for POI 16
Isochrones calculated for POI 17
Isochrones calculated for POI 18
Isochrones calculated for POI 19
Isochrones calculated for POI 20
Isochrones calculated for POI 21
Isochrones calculated for POI 22
Isochrones calculated for POI 23
Isochrones calculated for POI 24
Isochrones calculated for POI 25
Isochrones calculated for POI 26
Isochrones calculated for POI 27
Isochrones calculated for POI 28
Isochrones calculated for POI 29
Isochrones calculate

/var/folders/_n/_djf18r57q3482fkc8bbytk80000gn/T/ipykernel_52315/1740222986.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  poi['centroid'] = poi['geometry'].centroid


Isochrones calculated for POI 0
Isochrones calculated for POI 1
Isochrones calculated for POI 2
Isochrones calculated for POI 3
Isochrones calculated for POI 4
Isochrones calculated for POI 5
Isochrones calculated for POI 6
Isochrones calculated for POI 7
Isochrones calculated for POI 8
Isochrones calculated for POI 9
Isochrones calculated for POI 10
Isochrones calculated for POI 11
Isochrones calculated for POI 12
Isochrones calculated for POI 13
Isochrones calculated for POI 14
Isochrones calculated for POI 15
Isochrones calculated for POI 16
Isochrones calculated for POI 17
Isochrones calculated for POI 18
Isochrones calculated for POI 19
Isochrones calculated for POI 20
Isochrones calculated for POI 21
Isochrones calculated for POI 22
Isochrones calculated for POI 23
Isochrones calculated for POI 24
Isochrones calculated for POI 25
Isochrones calculated for POI 26
Isochrones calculated for POI 27
Isochrones calculated for POI 28
Isochrones calculated for POI 29
Isochrones calculate

/Users/celinathome/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/celinathome/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/celinathome/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/celinathome/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/celinatho

Isochrones calculated for POI 106
Isochrones calculated for POI 107
Isochrones calculated for POI 108


/Users/celinathome/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 6th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


Isochrones calculated for POI 109
Isochrones calculated for POI 110
Isochrones calculated for POI 111
Isochrones calculated for POI 112
Isochrones calculated for POI 113
Isochrones calculated for POI 114
Isochrones calculated for POI 115
Isochrones calculated for POI 116
Isochrones calculated for POI 117
Isochrones calculated for POI 118
Unable to build an isochrone map for POI. Skipping...
Isochrones calculated for POI 120
Isochrones calculated for POI 121
Isochrones calculated for POI 122
Isochrones calculated for POI 123
Isochrones calculated for POI 124
Isochrones calculated for POI 125
Isochrones calculated for POI 126
Isochrones calculated for POI 127
Isochrones calculated for POI 128
Isochrones calculated for POI 129
Isochrones calculated for POI 130
Isochrones calculated for POI 131
Isochrones calculated for POI 132
Isochrones calculated for POI 133
Isochrones calculated for POI 134
Isochrones calculated for POI 135
Isochrones calculated for POI 136
Isochrones calculated for PO

## Public spaces
### Barcelona

definitive wheelchair access

In [ ]:
input_file = '../data/output/barcelona/access/wheelchair_publicspaces.geojson'
output_path = '../data/output/barcelona/isochrones_wheelchair_public_spaces/definitive/'
output_filename = 'b_iso_wheelchair_public_spaces_d'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_wheelchair(input_file, output_path, output_filename, print_statement)

with nulls

In [ ]:
input_file = '../data/output/barcelona/access/combined_public_spaces_wheelchair.geojson'
output_path = '../data/output/barcelona/isochrones_wheelchair_public_spaces/might/'
output_filename = 'b_iso_wheelchair_public_spaces_m'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_wheelchair(input_file, output_path, output_filename, print_statement)

### Munich

In [ ]:
input_file = '../data/output/munich/access/wheelchair_publicspaces.geojson'
output_path = '../data/output/munich/isochrones_wheelchair_public_spaces/definitive/'
output_filename = 'm_iso_wheelchair_public_spaces_d'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_wheelchair(input_file, output_path, output_filename, print_statement)

In [ ]:
input_file = '../data/output/munich/access/combined_public_spaces_wheelchair.geojson'
output_path = '../data/output/munich/isochrones_wheelchair_public_spaces/might/'
output_filename = 'm_iso_wheelchair_public_spaces_m'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_wheelchair(input_file, output_path, output_filename, print_statement)

## Public building
### Barcelona

definitive wheelchair access

In [8]:
input_file = '../data/output/barcelona/access/wheelchair_public_building.geojson'
output_path = '../data/output/barcelona/isochrones_wheelchair_public_building/definitive/'
output_filename = 'b_iso_wheelchair_public_building_d'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_wheelchair(input_file, output_path, output_filename, print_statement)

/var/folders/_n/_djf18r57q3482fkc8bbytk80000gn/T/ipykernel_52315/1740222986.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  poi['centroid'] = poi['geometry'].centroid


Isochrones calculated for POI 0
Isochrones calculated for POI 1
Isochrones calculated for POI 2
Isochrones calculated for POI 3
Isochrones calculated for POI 4
Isochrones calculated for POI 5
Isochrones calculated for POI 6
Isochrones calculated for POI 7
Isochrones calculated for POI 8
Isochrones calculated for POI 9
Isochrones calculated for POI 10
Isochrones calculated for POI 11
Unable to build an isochrone map for POI. Skipping...
Isochrones calculated for POI 13
Isochrones calculated for POI 14
Unable to build an isochrone map for POI. Skipping...
Isochrones calculated for POI 16
Isochrones calculated for POI 17
Isochrones calculated for POI 18
Isochrones calculated for POI 19
Isochrones calculated for POI 20
Isochrones calculated for POI 21
Isochrones calculated for POI 22
Isochrones calculated for POI 23
Isochrones calculated for POI 24
Isochrones calculated for POI 25
Isochrones calculated for POI 26
Isochrones calculated for POI 27
Unable to build an isochrone map for POI. Sk

/Users/celinathome/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/celinathome/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/celinathome/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/celinathome/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/celinatho

Isochrones calculated for POI 29
Isochrones calculated for POI 30
Isochrones calculated for POI 31
Isochrones calculated for POI 32
Isochrone calculation completed.


with nulls

In [9]:
input_file = '../data/output/barcelona/access/combined_public_building_wheelchair.geojson'
output_path = '../data/output/barcelona/isochrones_wheelchair_public_building/might/'
output_filename = 'b_iso_wheelchair_public_building_m'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_wheelchair(input_file, output_path, output_filename, print_statement)

/var/folders/_n/_djf18r57q3482fkc8bbytk80000gn/T/ipykernel_52315/1740222986.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  poi['centroid'] = poi['geometry'].centroid


Isochrones calculated for POI 0
Isochrones calculated for POI 1
Isochrones calculated for POI 2
Isochrones calculated for POI 3
Isochrones calculated for POI 4
Isochrones calculated for POI 5
Isochrones calculated for POI 6
Isochrones calculated for POI 7
Isochrones calculated for POI 8
Isochrones calculated for POI 9
Isochrones calculated for POI 10
Isochrones calculated for POI 11
Unable to build an isochrone map for POI. Skipping...
Isochrones calculated for POI 13
Isochrones calculated for POI 14
Unable to build an isochrone map for POI. Skipping...
Isochrones calculated for POI 16
Isochrones calculated for POI 17
Isochrones calculated for POI 18
Isochrones calculated for POI 19
Isochrones calculated for POI 20
Isochrones calculated for POI 21
Isochrones calculated for POI 22
Isochrones calculated for POI 23
Isochrones calculated for POI 24
Isochrones calculated for POI 25
Isochrones calculated for POI 26
Isochrones calculated for POI 27
Unable to build an isochrone map for POI. Sk

/Users/celinathome/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/celinathome/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/celinathome/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/celinathome/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/celinatho

Isochrones calculated for POI 29
Isochrones calculated for POI 30
Isochrones calculated for POI 31
Isochrones calculated for POI 32
Isochrones calculated for POI 33
Isochrones calculated for POI 34
Isochrones calculated for POI 35
Isochrones calculated for POI 36
Isochrones calculated for POI 37
Isochrones calculated for POI 38
Isochrones calculated for POI 39
Isochrones calculated for POI 40
Isochrones calculated for POI 41
Isochrones calculated for POI 42
Isochrones calculated for POI 43
Isochrones calculated for POI 44
Isochrones calculated for POI 45
Isochrones calculated for POI 46
Isochrones calculated for POI 47
Isochrones calculated for POI 48
Isochrones calculated for POI 49
Isochrones calculated for POI 50
Isochrones calculated for POI 51
Isochrones calculated for POI 52
Isochrones calculated for POI 53
Isochrones calculated for POI 54
Isochrones calculated for POI 55
Isochrones calculated for POI 56
Isochrones calculated for POI 57
Isochrones calculated for POI 58
Isochrones

Isochrones calculated for POI 268
Isochrones calculated for POI 269
Isochrones calculated for POI 270
Isochrones calculated for POI 271
Isochrones calculated for POI 272
Isochrones calculated for POI 273
Isochrones calculated for POI 274
Isochrones calculated for POI 275
Isochrones calculated for POI 276
Isochrones calculated for POI 277
Isochrones calculated for POI 278
Isochrones calculated for POI 279
Isochrones calculated for POI 280
Isochrones calculated for POI 281
Isochrones calculated for POI 282
Isochrones calculated for POI 283
Isochrones calculated for POI 284
Isochrones calculated for POI 285
Isochrones calculated for POI 286
Isochrones calculated for POI 287
Isochrones calculated for POI 288
Isochrones calculated for POI 289
Isochrones calculated for POI 290
Isochrones calculated for POI 291
Isochrones calculated for POI 292
Isochrones calculated for POI 293
Isochrones calculated for POI 294
Isochrones calculated for POI 295
Isochrones calculated for POI 296
Isochrones cal

/Users/celinathome/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 8th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


Isochrones calculated for POI 362
Isochrones calculated for POI 363
Isochrones calculated for POI 364
Isochrones calculated for POI 365
Isochrones calculated for POI 366
Isochrones calculated for POI 367
Isochrones calculated for POI 368
Isochrones calculated for POI 369
Isochrones calculated for POI 370
Isochrones calculated for POI 371
Isochrones calculated for POI 372
Isochrones calculated for POI 373
Isochrones calculated for POI 374
Isochrones calculated for POI 375
Isochrones calculated for POI 376
Isochrones calculated for POI 377
Isochrones calculated for POI 378
Isochrones calculated for POI 379
Isochrones calculated for POI 380
Isochrones calculated for POI 381
Isochrones calculated for POI 382
Unable to build an isochrone map for POI. Skipping...
Isochrones calculated for POI 384
Isochrones calculated for POI 385
Isochrones calculated for POI 386
Isochrones calculated for POI 387
Isochrones calculated for POI 388
Isochrones calculated for POI 389
Isochrones calculated for PO

Isochrones calculated for POI 597
Isochrones calculated for POI 598
Isochrones calculated for POI 599
Isochrones calculated for POI 600
Isochrones calculated for POI 601
Isochrones calculated for POI 602
Unable to build an isochrone map for POI. Skipping...
Isochrones calculated for POI 604
Isochrones calculated for POI 605
Isochrones calculated for POI 606
Isochrones calculated for POI 607
Isochrones calculated for POI 608
Isochrones calculated for POI 609
Isochrones calculated for POI 610
Isochrones calculated for POI 611
Isochrones calculated for POI 612
Isochrones calculated for POI 613
Isochrones calculated for POI 614
Isochrones calculated for POI 615
Isochrones calculated for POI 616
Isochrones calculated for POI 617
Isochrones calculated for POI 618
Isochrones calculated for POI 619
Isochrones calculated for POI 620
Isochrones calculated for POI 621
Isochrones calculated for POI 622
Isochrones calculated for POI 623
Isochrones calculated for POI 624
Isochrones calculated for PO

Isochrones calculated for POI 835
Isochrones calculated for POI 836
Isochrones calculated for POI 837
Isochrones calculated for POI 838
Isochrones calculated for POI 839
Isochrones calculated for POI 840
Isochrones calculated for POI 841
Isochrones calculated for POI 842
Isochrones calculated for POI 843
Isochrones calculated for POI 844
Isochrones calculated for POI 845
Isochrones calculated for POI 846
Isochrones calculated for POI 847
Isochrones calculated for POI 848
Isochrones calculated for POI 849
Isochrones calculated for POI 850
Isochrones calculated for POI 851
Isochrones calculated for POI 852
Isochrones calculated for POI 853
Isochrones calculated for POI 854
Isochrones calculated for POI 855
Isochrones calculated for POI 856
Isochrones calculated for POI 857
Isochrones calculated for POI 858
Isochrones calculated for POI 859
Isochrones calculated for POI 860
Isochrones calculated for POI 861
Isochrones calculated for POI 862
Isochrones calculated for POI 863
Isochrones cal

### Munich

In [10]:
input_file = '../data/output/munich/access/wheelchair_public_building.geojson'
output_path = '../data/output/munich/isochrones_wheelchair_public_building/definitive/'
output_filename = 'm_iso_wheelchair_public_building_d'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_wheelchair(input_file, output_path, output_filename, print_statement)

/var/folders/_n/_djf18r57q3482fkc8bbytk80000gn/T/ipykernel_52315/1740222986.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  poi['centroid'] = poi['geometry'].centroid


Isochrones calculated for POI 0
Isochrones calculated for POI 1
Isochrones calculated for POI 2
Isochrones calculated for POI 3
Isochrones calculated for POI 4
Isochrones calculated for POI 5
Isochrones calculated for POI 6
Isochrones calculated for POI 7
Isochrones calculated for POI 8
Isochrones calculated for POI 9
Isochrones calculated for POI 10
Isochrones calculated for POI 11
Isochrones calculated for POI 12
Isochrones calculated for POI 13
Isochrones calculated for POI 14
Isochrones calculated for POI 15
Isochrones calculated for POI 16
Isochrones calculated for POI 17
Isochrones calculated for POI 18
Isochrones calculated for POI 19
Isochrones calculated for POI 20
Isochrones calculated for POI 21
Isochrones calculated for POI 22
Isochrones calculated for POI 23
Isochrones calculated for POI 24
Isochrones calculated for POI 25
Isochrones calculated for POI 26
Isochrones calculated for POI 27
Isochrones calculated for POI 28
Isochrones calculated for POI 29
Isochrones calculate

In [11]:
input_file = '../data/output/munich/access/combined_public_building_wheelchair.geojson'
output_path = '../data/output/munich/isochrones_wheelchair_public_building/might/'
output_filename = 'm_iso_wheelchair_public_building_m'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_wheelchair(input_file, output_path, output_filename, print_statement)

/var/folders/_n/_djf18r57q3482fkc8bbytk80000gn/T/ipykernel_52315/1740222986.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  poi['centroid'] = poi['geometry'].centroid


Isochrones calculated for POI 0
Isochrones calculated for POI 1
Isochrones calculated for POI 2
Isochrones calculated for POI 3
Isochrones calculated for POI 4
Isochrones calculated for POI 5
Isochrones calculated for POI 6
Isochrones calculated for POI 7
Isochrones calculated for POI 8
Isochrones calculated for POI 9
Isochrones calculated for POI 10
Isochrones calculated for POI 11
Isochrones calculated for POI 12
Isochrones calculated for POI 13
Isochrones calculated for POI 14
Isochrones calculated for POI 15
Isochrones calculated for POI 16
Isochrones calculated for POI 17
Isochrones calculated for POI 18
Isochrones calculated for POI 19
Isochrones calculated for POI 20
Isochrones calculated for POI 21
Isochrones calculated for POI 22
Isochrones calculated for POI 23
Isochrones calculated for POI 24
Isochrones calculated for POI 25
Isochrones calculated for POI 26
Isochrones calculated for POI 27
Isochrones calculated for POI 28
Isochrones calculated for POI 29
Isochrones calculate

/Users/celinathome/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/celinathome/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/celinathome/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/celinathome/anaconda3/lib/python3.11/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/celinatho

Isochrones calculated for POI 119
Isochrones calculated for POI 120
Isochrones calculated for POI 121
Isochrones calculated for POI 122
Isochrones calculated for POI 123
Isochrones calculated for POI 124
Isochrones calculated for POI 125
Isochrones calculated for POI 126
Isochrones calculated for POI 127
Isochrones calculated for POI 128
Isochrones calculated for POI 129
Isochrones calculated for POI 130
Isochrones calculated for POI 131
Isochrones calculated for POI 132
Isochrones calculated for POI 133
Isochrones calculated for POI 134
Isochrones calculated for POI 135
Isochrones calculated for POI 136
Isochrones calculated for POI 137
Isochrones calculated for POI 138
Isochrones calculated for POI 139
Isochrones calculated for POI 140
Isochrones calculated for POI 141
Isochrones calculated for POI 142
Isochrones calculated for POI 143
Isochrones calculated for POI 144
Isochrones calculated for POI 145
Isochrones calculated for POI 146
Isochrones calculated for POI 147
Isochrones cal

# Walking Isochrones

In [ ]:
import json
import time
import os
import geopandas as gpd
from openrouteservice import Client
from openrouteservice.exceptions import ApiError

def calculate_isochrones_walk(input_file, output_path, output_filename, print_statement):
    # Load poi data with GeoPandas
    poi = gpd.read_file(input_file)

    # Calculate centroids for each polygon
    poi['centroid'] = poi['geometry'].centroid

    # Change CRS to EPSG:4326
    poi = poi.to_crs("EPSG:4326")
    
    # Get the name of the ID column
    poi_id_column = 'id'  # Assuming the column name is 'id'

    # Initialize OpenRouteService client
    api_key = "5b3ce3597851110001cf6248ec8fc0286b4a4fd1a90fc8108327d823"  # Replace with your actual API key
    client = Client(key=api_key)

    # Define parameters for isochrone calculation
    profile = 'foot-walking'
    range_type = 'time'
    range_seconds = [900, 1800]  # 15 and 30 Minutes

    # Create the directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)

    # Function to calculate isochrones with retry mechanism
    def calculate_isochrones_with_retry(coordinate):
        retries = 3  # Number of retries
        for attempt in range(retries):
            try:
                isochrones = client.isochrones(locations=[coordinate], profile=profile, range_type=range_type, range=range_seconds, attributes=['total_pop'])
                return isochrones
            except ApiError as e:
                if "rate limit exceeded" in str(e):
                    print(f"Rate limit exceeded. Retrying in 10 seconds...")
                    time.sleep(10)  # Retry after 10 seconds for rate limit errors
                elif "Unable to build an isochrone map" in str(e):
                    print(f"Unable to build an isochrone map for POI. Skipping...")
                    return None
                else:
                    raise e  # Re-raise the exception if it's not related to rate limit exceeded
        return None

    # Iterate over poi and calculate isochrones
    for idx, row in poi.iterrows():
        # Get centroid coordinates of the poi
        lon, lat = row['centroid'].x, row['centroid'].y
        coordinate = (lon, lat)

        # Calculate isochrones with retry mechanism
        isochrones = calculate_isochrones_with_retry(coordinate)
        if isochrones is None:
            continue  # Skip the poi if isochrones cannot be calculated
            
        # Get the poi ID from the column
        poi_id = row[poi_id_column]            

        # Add poi ID to each isochrone feature
        for feature in isochrones['features']:
            feature['properties']['poi_id'] = poi_id

        # Save isochrones data to file
        output_file_path = os.path.join(output_path, f'{output_filename}_{idx + 1}.geojson')
        
        with open(output_file_path, 'w') as output_file:
            json.dump(isochrones, output_file)

        print(print_statement.format(idx))

        # Rate limit handling
        time.sleep(3)  # Add a small delay to avoid rate limits and respect the rate limit

    print("Isochrone calculation completed.")


## Green spaces
### Barcelona

definitive wheelchair

In [ ]:
input_file = '../data/output/barcelona/access/wheelchair_greenspace.geojson'
output_path = '../data/output/barcelona/isochrones_foot_greenspaces_wheelchair/'
output_filename = 'b_iso_foot_green_w'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_walk(input_file, output_path, output_filename, print_statement)

with ramps

In [ ]:
input_file = '../data/output/barcelona/access/combined_greenspaces_wheelchair.geojson'
output_path = '../data/output/barcelona/isochrones_foot_greenspaces/'
output_filename = 'b_iso_foot_green'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_walk(input_file, output_path, output_filename, print_statement)

### Munich

In [ ]:
input_file = '../data/output/munich/access/wheelchair_greenspace.geojson'
output_path = '../data/output/munich/isochrones_foot_greenspaces_wheelchair/'
output_filename = 'b_iso_foot_green_w'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_walk(input_file, output_path, output_filename, print_statement)

In [ ]:
input_file = '../data/output/munich/access/combined_greenspaces_wheelchair.geojson'
output_path = '../data/output/munich/isochrones_foot_greenspaces/'
output_filename = 'b_iso_foot_green'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_walk(input_file, output_path, output_filename, print_statement)

## Public spaces
### Barcelona

definitive wheelchair

In [ ]:
input_file = '../data/output/barcelona/access/wheelchair_publicspaces.geojson'
output_path = '../data/output/barcelona/isochrones_foot_public_spaces_wheelchair/'
output_filename = 'b_iso_foot_public_spaces_w'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_walk(input_file, output_path, output_filename, print_statement)

with nulls

In [ ]:
input_file = '../data/output/barcelona/access/combined_public_spaces_wheelchair.geojson'
output_path = '../data/output/barcelona/isochrones_foot_public_spaces/'
output_filename = 'b_iso_foot_public_spaces'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_walk(input_file, output_path, output_filename, print_statement)

### Munich

In [ ]:
input_file = '../data/output/munich/access/wheelchair_publicspaces.geojson'
output_path = '../data/output/munich/isochrones_foot_public_spaces_wheelchair/'
output_filename = 'b_iso_foot_public_spaces_w'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_walk(input_file, output_path, output_filename, print_statement)


In [ ]:
input_file = '../data/output/munich/access/combined_public_spaces_wheelchair.geojson'
output_path = '../data/output/munich/isochrones_foot_public_spaces/'
output_filename = 'b_iso_foot_public_spaces'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_walk(input_file, output_path, output_filename, print_statement)

## Public buildings
### Barcelona

definitive wheelchair

In [ ]:
input_file = '../data/output/barcelona/access/wheelchair_public_building.geojson'
output_path = '../data/output/barcelona/isochrones_foot_public_building_wheelchair/'
output_filename = 'b_iso_foot_public_building_w'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_walk(input_file, output_path, output_filename, print_statement)

with nulls

In [ ]:
input_file = '../data/output/barcelona/access/combined_public_building_wheelchair.geojson'
output_path = '../data/output/barcelona/isochrones_foot_public_building_wheelchair/'
output_filename = 'b_iso_foot_public_building_w'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_walk(input_file, output_path, output_filename, print_statement)

### Munich

In [ ]:
input_file = '../data/output/munich/access/wheelchair_public_building.geojson'
output_path = '../data/output/munich/isochrones_foot_public_building_wheelchair/'
output_filename = 'b_iso_foot_public_building_w'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_walk(input_file, output_path, output_filename, print_statement)

In [ ]:
input_file = '../data/output/munich/access/combined_public_building_wheelchair.geojson'
output_path = '../data/output/munich/isochrones_foot_public_building_wheelchair/'
output_filename = 'b_iso_foot_public_building_w'
print_statement = "Isochrones calculated for POI {}"

calculate_isochrones_walk(input_file, output_path, output_filename, print_statement)